In [ ]:
import os
import pandas as pd
import re

# Load all downloaded data

In [ ]:
data_dct = {}
for file in os.listdir("./Data"):
    dataset_name = re.match("^.*-(.*).csv$", file).group(1)
    df = pd.read_csv(f"./Data/{file}")
    if "DATE" in df.columns:
        df = df.rename({"DATE": "Date"}, axis=1)
    df["Date"] = pd.to_datetime(df["Date"])
    data_dct[dataset_name] = df

In [ ]:
data_dct.keys()

# Preprocessing 

## Aggregate daily to monthly (first day of month)

In [ ]:
yield_monthly_df = (data_dct["YIELD"]
                    .assign(Year = lambda x: x["Date"].dt.year, Month = lambda x: x["Date"].dt.month)
                    .astype({"Month": str})
                    .groupby(["Year", "Month"])
                    .mean(numeric_only=True)
                    .reset_index()
                   )
yield_monthly_df["Date"] = df.apply(lambda x: pd.to_datetime(f"{x.Year}-{(x.Month.zfill(2))}-01", format="%Y-%m-%d"), axis=1)

In [ ]:
yield_monthly_df.groupby(["Year"]).agg(lambda x: sum(pd.isna(x)))

In [ ]:
# data_dct["YIELD"].query("Date >= '2017-01-01' & Date <= '2017-12-31' & `2 MO`.isnull()")

# Merge to form dataset

In [ ]:
merged_df = (
        data_dct["SP500_REAL_PRICE_MONTH"].rename({"Value": "PRICE"}, axis=1)
          .merge(yield_monthly_df, on="Date", how="outer")
          .merge(data_dct["EARNINGS_YIELD_MONTH"].rename({"Value": "EARNY"}, axis=1),
                 on="Date", how="outer")
          .merge(data_dct["SP500_DIV_MONTH"].rename({"Value": "DIV"}, axis=1),
                 on="Date", how="outer")
          .merge(data_dct["SP500_PBV_RATIO_QUARTER"].rename({"Value": "PBV"}, axis=1),
                 on="Date", how="outer")
          .merge(data_dct["SP500_PE_RATIO_MONTH"].rename({"Value": "PE"}, axis=1),
                 on="Date", how="outer")
          .merge(data_dct["SP500_PSR_QUARTER"].rename({"Value": "PSR"}, axis=1),
                 on="Date", how="outer")
          .merge(data_dct["SOC1"].rename({"Index": "SOC1"}, axis=1),
                 on="Date", how="outer")
          .sort_values("Date")
          .reset_index(drop=True)
          .assign(Year = lambda x: x["Date"].dt.year, 
                  Month = lambda x: x["Date"].dt.month)
         )

In [ ]:
data_df = (merged_df
           .groupby(["Year", "Month"])
           .head(1)
          )

In [ ]:
(data_df
 .query("Date >= '1990-01-01'")
 .isna()
 .sum(axis=0)
)

In [ ]:
# select a subset where there are no NaNs for preliminary modelling
(data_df
 .query("Date >= '1990-01-01'")
 [["Date", "PRICE", "EARNY", "PE", "3 MO", "6 MO", "1 YR", "2 YR", "3 YR", "5 YR", "7 YR", "10 YR"]]
 .to_csv("forecasting_data.csv", index=False)
)